https://wiseodd.github.io/techblog/2016/09/17/gan-tensorflow/

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


X = tf.placeholder(tf.float32, shape=[None, 784])

D_W1 = tf.Variable(xavier_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


Z = tf.placeholder(tf.float32, shape=[None, 100])

G_W1 = tf.Variable(xavier_init([100, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(xavier_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1, G_W2, G_b1, G_b2]


def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob


def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit


def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig


G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

# D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
# G_loss = -tf.reduce_mean(tf.log(D_fake))

# Alternative losses:
# -------------------
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

mb_size = 128
Z_dim = 100

mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0

for it in range(1000000):
    if it % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz
Iter: 0
D loss: 1.569
G_loss: 2.294

Iter: 1000
D loss: 0.005583
G_loss: 8.572

Iter: 2000
D loss: 0.06688
G_loss: 4.551

Iter: 3000
D loss: 0.08497
G_loss: 5.084

Iter: 4000
D loss: 0.1144
G_loss: 4.941

Iter: 5000
D loss: 0.3299
G_loss: 5.298

Iter: 6000
D loss: 0.401
G_loss: 3.711

Iter: 7000
D loss: 0.4147
G_loss: 4.07

Iter: 8000
D loss: 0.3481
G_loss: 4.078

Iter: 9000
D loss: 0.5323
G_loss: 3.775

Iter: 10000
D loss: 0.5761
G_loss: 2.693

Iter: 11000
D loss: 0.7234
G_loss: 2.751

Iter: 12000
D loss: 0.6901
G_loss

Iter: 184000
D loss: 0.5409
G_loss: 2.684

Iter: 185000
D loss: 0.5132
G_loss: 2.883

Iter: 186000
D loss: 0.5306
G_loss: 2.797

Iter: 187000
D loss: 0.4097
G_loss: 2.508

Iter: 188000
D loss: 0.509
G_loss: 3.052

Iter: 189000
D loss: 0.5811
G_loss: 2.745

Iter: 190000
D loss: 0.5389
G_loss: 2.484

Iter: 191000
D loss: 0.6751
G_loss: 2.708

Iter: 192000
D loss: 0.6412
G_loss: 2.604

Iter: 193000
D loss: 0.4652
G_loss: 2.496

Iter: 194000
D loss: 0.5828
G_loss: 2.597

Iter: 195000
D loss: 0.5716
G_loss: 3.063

Iter: 196000
D loss: 0.461
G_loss: 2.706

Iter: 197000
D loss: 0.4707
G_loss: 2.457

Iter: 198000
D loss: 0.4799
G_loss: 2.546

Iter: 199000
D loss: 0.4075
G_loss: 2.948

Iter: 200000
D loss: 0.6089
G_loss: 2.668

Iter: 201000
D loss: 0.4937
G_loss: 2.821

Iter: 202000
D loss: 0.4136
G_loss: 2.546

Iter: 203000
D loss: 0.5484
G_loss: 2.944

Iter: 204000
D loss: 0.5295
G_loss: 2.52

Iter: 205000
D loss: 0.4275
G_loss: 2.835

Iter: 206000
D loss: 0.536
G_loss: 2.509

Iter: 207000
D 

Iter: 376000
D loss: 0.5113
G_loss: 2.751

Iter: 377000
D loss: 0.4584
G_loss: 2.784

Iter: 378000
D loss: 0.3873
G_loss: 2.977

Iter: 379000
D loss: 0.3951
G_loss: 3.056

Iter: 380000
D loss: 0.4988
G_loss: 2.91

Iter: 381000
D loss: 0.3702
G_loss: 2.88

Iter: 382000
D loss: 0.3445
G_loss: 3.071

Iter: 383000
D loss: 0.5711
G_loss: 2.963

Iter: 384000
D loss: 0.5331
G_loss: 2.947

Iter: 385000
D loss: 0.3937
G_loss: 2.613

Iter: 386000
D loss: 0.463
G_loss: 2.761

Iter: 387000
D loss: 0.3365
G_loss: 2.829

Iter: 388000
D loss: 0.4653
G_loss: 3.115

Iter: 389000
D loss: 0.4205
G_loss: 2.552

Iter: 390000
D loss: 0.4804
G_loss: 3.049

Iter: 391000
D loss: 0.4384
G_loss: 3.181

Iter: 392000
D loss: 0.4371
G_loss: 3.238

Iter: 393000
D loss: 0.4714
G_loss: 2.96

Iter: 394000
D loss: 0.4465
G_loss: 2.931

Iter: 395000
D loss: 0.3754
G_loss: 3.161

Iter: 396000
D loss: 0.3885
G_loss: 2.683

Iter: 397000
D loss: 0.5671
G_loss: 2.922

Iter: 398000
D loss: 0.4803
G_loss: 3.036

Iter: 399000
D 

Iter: 568000
D loss: 0.3951
G_loss: 2.916

Iter: 569000
D loss: 0.5255
G_loss: 3.351

Iter: 570000
D loss: 0.2758
G_loss: 3.382

Iter: 571000
D loss: 0.4014
G_loss: 3.184

Iter: 572000
D loss: 0.3997
G_loss: 3.062

Iter: 573000
D loss: 0.4849
G_loss: 3.608

Iter: 574000
D loss: 0.346
G_loss: 2.867

Iter: 575000
D loss: 0.3381
G_loss: 2.964

Iter: 576000
D loss: 0.3389
G_loss: 3.776

Iter: 577000
D loss: 0.4047
G_loss: 3.139

Iter: 578000
D loss: 0.3628
G_loss: 3.74

Iter: 579000
D loss: 0.351
G_loss: 3.099

Iter: 580000
D loss: 0.3379
G_loss: 3.337

Iter: 581000
D loss: 0.4193
G_loss: 3.756

Iter: 582000
D loss: 0.4425
G_loss: 3.115

Iter: 583000
D loss: 0.4882
G_loss: 3.931

Iter: 584000
D loss: 0.5499
G_loss: 3.43

Iter: 585000
D loss: 0.3519
G_loss: 3.604

Iter: 586000
D loss: 0.4736
G_loss: 3.414

Iter: 587000
D loss: 0.4069
G_loss: 3.774

Iter: 588000
D loss: 0.2792
G_loss: 3.275

Iter: 589000
D loss: 0.4776
G_loss: 2.994

Iter: 590000
D loss: 0.381
G_loss: 3.173

Iter: 591000
D l

In [2]:
print(tf.__version__)

1.3.0
